<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Уравнения Эйлера для невязкого течения

### Examples - Fluid Mechanics
<section class="post-meta">
By Knut Sverdrup, October 2016
</section>
Last edited: March 22nd 2018
___

## Вступление 

Этот урок представляет собой введение в набор уравнений в частных производных, которые широко используются для моделирования аэродинамики, атмосферы и климата, взрывных детонаций и даже астрофизики. Здесь дано лишь небольшое представление о мире гиперболических PDE, задач Римана и вычислительной гидродинамики, и заинтересованному читателю рекомендуется продолжить изучение этой области [1]. 

Уравнения Эйлера управляют адиабатическим и невязким течением жидкости. В пределе Фруда (без внешних сил тела) в одном измерении, с плотностью $\rho$, скоростью $u$, полной энергией $E$ и давлением $p$, они даны в безразмерной форме как

\begin{align*}
    \frac{\partial \rho}{\partial t} + \frac{\partial (\rho u)}{\partial x} &= 0, \\
    \frac{\partial (\rho u)}{\partial t} + \frac{\partial (\rho u^2 + p)}{\partial x} &= 0, \\
    \frac{\partial (\rho E)}{\partial t} + \frac{\partial u(E + p)}{\partial x} &= 0.
\end{align*}

Эти три уравнения описывают сохранение массы, импульса и энергии соответственно.
Чтобы решить их численно, мы начинаем с импорта NumPy и настройки среды построения графиков. 

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
newparams = {'font.size': 14, 'figure.figsize': (14, 7),
             'mathtext.fontset': 'stix', 'font.family': 'STIXGeneral',
             'lines.linewidth': 2}
plt.rcParams.update(newparams)

## Идеальные газы


Для идеального газа общая энергия представляет собой сумму кинетического и потенциального вкладов, т.е. 

\begin{equation*}
    E = \frac{1}{2} \rho u^2 + \frac{p}{\gamma - 1}, 
\end{equation*}

где $\gamma$ - это соотношение удельных температур для материала в нашей системе. Мы будем рассматривать воздух, для которого $\gamma = 1.4$. Обратите внимание, что $e = \frac{p}{(\gamma - 1)\rho}$ - это удельная внутренняя энергия для идеальных газов. Преобразования между энергией и давлением будут полезны нам позже, поэтому мы определим соответствующие функции:

In [ ]:
def energy(rho, u, p):
    return 0.5 * rho * u ** 2 + p / 0.4 

def pressure(rho, u, E):
    return 0.4 * (E - 0.5 * rho * u ** 2)

## Законы сохранения

Удобно, что эта гиперболическая система PDE первого порядка может быть записана как набор законов сохранения, т.е.

\begin{equation*}
    \partial_t {\bf Q} + \partial_x {\bf F(Q)} = {\bf 0}\,.
\end{equation*}

Здесь вектор сохраняемых величин ${\bf Q}$ и их потоки ${\bf F(Q)}$ задаются

\begin{equation*}
    {\bf Q} = 
    \begin{bmatrix} 
        \rho \\ 
        \rho u \\ 
        E 
    \end{bmatrix}
    \,,\quad
    {\bf F} = 
    \begin{bmatrix}
        \rho u \\ 
        \rho u^2 + p \\
        u(E+p) 
    \end{bmatrix}
    \,.
\end{equation*}

Учитывая состояние системы (т.е. вектор сохраняемых величин), мы вычисляем поток как: 

In [ ]:
def flux(Q):
    rho, u, E = Q[0], Q[1] / Q[0], Q[2]
    p = pressure(rho, u, E)
    F = np.empty_like(Q)
    F[0] = rho * u
    F[1] = rho * u ** 2 + p
    F[2] = u * (E + p)
    return F

## Конечные объемы

Рассмотрим пространственную область $[x_L, x_R]$ и две точки во времени $t_2 > t_1$. Интегрируя уравнения Эйлера в дифференциальной форме в пространстве и времени, мы получаем интегральную форму, 

\begin{equation*}
    \int_{x_L}^{x_R} {\bf Q}(x, t_2) {\rm d} x = \int_{x_L}^{x_R} {\bf Q}(x, t_1) {\rm d} x 
    + \int_{t_1}^{t_2} {\bf F}({\bf Q}(x_L, t)) {\rm d} t - \int_{t_1}^{t_2} {\bf F}({\bf Q}(x_R, t)) {\rm d} t .
\end{equation*}

Это соотношение является основой для нашей пространственной и временной дискретизации. Для простоты мы принимаем нашу вычислительную область равной $[0, 1]$ и делим ее на $N$ равных ячеек шириной $\Delta x = 1/N$: 

In [ ]:
N = 100
dx = 1 / N
x = np.linspace(-0.5 * dx, 1 + 0.5 * dx, N + 2)

Обратите внимание, что мы добавили по одной дополнительной ячейке на каждую сторону домена. Это так называемые призрачные ячейки, которые позволяют нам применять соответствующие граничные условия. Также необходимо дискретизировать вектор состояния $\bf Q$ и межклеточные потоки $\bf F$. Обозначим через ${\bf Q}_i^n$ среднее пространственное значение в ячейке $[x_{i-1/2}, x_{i+1/2}]$ в момент времени $t_n$, т. е.

\begin{equation*} 
    {\bf Q}_i^n = \frac{1}{\Delta x} \int_{x_{i-1/2}}^{x_{i+1/2}} {\bf Q}(x, t_n) {\rm d} x ,
\end{equation*}

и инициализируем массив numpy для хранения значений в каждой ячейке: 

In [ ]:
Q = np.empty((3, len(x))) 

Аналогично, среднее значение по времени потока через границу ячейки в $x_{i+1/2}$ обозначается ${\bf F}_{i+1/2}^n$: 

\begin{equation*}
    {\bf F}_{i+1/2}^n = \frac{1}{\Delta t^n} \int_{t_n}^{t_{n+1}} {\bf F}({\bf Q}(x_{i+1/2}, t)) {\rm d} t .
\end{equation*}

Вставляя эти дискретизации в интегральную форму уравнений Эйлера, мы получаем консервативную формулу обновления для каждой вычислительной ячейки, которая является точной: 

\begin{equation*}
    {\bf Q}_i^{n+1} = {\bf Q}_i^n + 
    \frac{\Delta t^n}{\Delta x} \left( {\bf F}_{i-\frac{1}{2}}^n - {\bf F}_{i+\frac{1}{2}}^n \right) 
\end{equation*}

Эта формула является нашим методом продвижения системы вперед во времени, и численные приближения заключаются исключительно в оценках межклеточных потоков ${\bf F}_{i \pm 1/2}^n$. Однако мы не можем выбрать временной шаг $\Delta t^n$ настолько большим, насколько мы хотим, из-за ограничений на стабильность. Задав коэффициент Куранта-Фридрихса-Льюиса (CFL) $c \leq 1$, шаг по времени можно безопасно установить равным

\begin{equation*}
    \Delta t^n = \frac{c \Delta x}{S_{\rm max}^n}, 
\end{equation*}

где $S_{\rm max}^n$ - это мера максимальной скорости волны, присутствующей в системе. Мы используем общее приближение, которое находит ячейку с наибольшей суммой скоростей материала и звука, т.е. 

\begin{equation*}
    S_{\rm max}^n = \max_i ( |u_i^n| + a_i^n ) , 
\end{equation*}

где скорость звука для идеальных газов определяется 

\begin{equation*}
    a = \sqrt{\frac{\gamma p}{\rho}} .
\end{equation*}

In [ ]:
def timestep(Q, c, dx):
    rho, u, E = Q[0], Q[1] / Q[0], Q[2]
    
    a = np.sqrt(1.4 * pressure(rho, u, E) / rho)
    S_max = np.max(np.abs(u) + a)
    
    return  c * dx / S_max

## FORCE схема для аппроксимации потока

Существует множество различных процедур для аппроксимации межклеточных потоков ${\bf F}_{i \pm 1/2}$. Для простоты мы реализуем относительно прямолинейную схему центрирования первого порядка (FIrst-ORder CEntred (FORCE)). [2] Учитывая состояние двух соседних ячеек, поток на границе раздела вычисляется как

\begin{equation*}
    {\bf F}_{\rm FORCE} ({\bf Q}_L, {\bf Q}_R) 
        =\frac{1}{2} \left( {\bf F}_0 + \frac{1}{2} ({\bf F}_L + {\bf F}_R) \right) + \frac{1}{4} \frac{\Delta x}{\Delta t^n} ({\bf Q}_L - {\bf Q}_R) . 
\end{equation*}

Здесь, ${\bf F}_K = {\bf F}({\bf Q}_K)$ и 

\begin{equation*}
    {\bf Q}_0 = \frac{1}{2} ({\bf Q}_L + {\bf Q}_R) + \frac{1}{2} \frac{\Delta t^n}{\Delta x} ({\bf F}_L- {\bf F}_R) , 
\end{equation*}

Эти уравнения соответствуют уравнениям (16) и (19) в [2]. 

В этот момент становится очевидной причина наших ранее введенных призрачных клеток. Поскольку интерфейс каждой ячейки требует информации с обеих сторон, первой и N-й ячейкам в нашем домене не хватает информации слева и справа соответственно. Инициализируя призрачную ячейку с каждой стороны и передавая информацию из области наружу для каждого временного шага, эти вымышленные ячейки предоставляют необходимую информацию для выполнения наших вычислений. 

Реализуя схему FORCE в Python, мы имеем 

In [ ]:
def force(Q):
    Q_L = Q[:, :-1]
    Q_R = Q[:, 1:]
    
    F_L = flux(Q_L)
    F_R = flux(Q_R)
    
    Q_0 = 0.5 * (Q_L + Q_R) + 0.5 * dt / dx * (F_L - F_R) 
    F_0 = flux(Q_0)
    
    return 0.5 * (F_0 + 0.5 * (F_L + F_R)) + 0.25 * dx / dt * (Q_L - Q_R)

## Ударная трубка Сода

Учитывая начальное условие ${\bf Q}(x, 0)$, мы хотим развить систему во времени, чтобы предсказать 
состояние в какое-то будущее время $t=T$. Популярным тестовым случаем для уравнений Эйлера является ударная трубка Сода. Тест состоит из задачи Римана, что означает, что PDE связан с набором кусочно-постоянных начальных условий, разделенных одним разрывом. Интуитивно, тест можно представить как трубку с мембраной, разделяющей воздух двух разных плотностей (и давлений). При $t=0$ мембрана удаляется, что приводит к волне разрежения, разрыву контакта и ударной волне. Начальные условия задаются

\begin{equation*}
    {\bf Q}(x, 0) = 
        \begin{cases} 
            {\bf Q}_L \quad {\rm if} \quad x \leq 0.5 \\ 
            {\bf Q}_R \quad {\rm if} \quad x > 0.5 
        \end{cases} , \quad 
    \begin{pmatrix} \rho \\ u \\ p \end{pmatrix}_L 
    = \begin{pmatrix} 1 \\ 0 \\ 1 \end{pmatrix} , \quad
    \begin{pmatrix} \rho \\ u \\ p \end{pmatrix}_R
    = \begin{pmatrix} 0.125 \\ 0 \\ 0.1 \end{pmatrix} .
\end{equation*}

Поэтому мы инициализируем вектор сохраняемых переменных в соответствии с 

In [ ]:
# Density: 
Q[0, x <= 0.5] = 1.0
Q[0, x > 0.5] = 0.125
# Momentum: 
Q[1] = 0.0
# Energy: 
Q[2, x <= 0.5] = energy(1.0, 0.0, 1.0)
Q[2, x > 0.5] = energy(0.125, 0.0, 0.1)

## Эволюция системы

Сейчас созданы все предпосылки для того, чтобы мы смогли разработать систему ударных трубок во времени численно. Мы выбираем коэффициент CFL 0.95 и переходим к конечному времени $T=0.25$

In [ ]:
c = 0.9
T = 0.25
t = 0
while t < T:
    # Compute time step size
    dt = timestep(Q, c, dx)
    if t + dt > T: # Make sure to end up at specified final time
        dt = T - t

    # Transmissive boundary conditions
    Q[:, 0] = Q[:, 1] # Left boundary 
    Q[:, N + 1] = Q[:, N] # Right boundary

    # Flux computations using FORCE scheme
    F = force(Q)

    # Conservative update formula
    Q[:, 1:-1] += dt / dx * (F[:, :-1] - F[:, 1:]) 
    
    # Go to next time step
    t += dt

## Результаты

Для сравнения наших результатов в файле приведено точное эталонное решение "ref.txt". Для простого случая единичного контактного разрыва эта задача Римана может быть решена с произвольной точностью с помощью точного (итерационного) решателя. Реализация точного решателя выходит за рамки этой тетради, но заинтересованный читатель обращается к обширному ресурсу Toro [1]. 

In [ ]:
# Load reference solution
ref_sol = np.loadtxt('ref.txt')
ref_sol = np.transpose(ref_sol)

# Numerical results for density, velocity, pressure and internal energy
num_sol = [Q[0], Q[1] / Q[0], pressure(Q[0], Q[1] / Q[0], Q[2]), \
    pressure(Q[0], Q[1] / Q[0], Q[2]) / (0.4 * Q[0])]

Имея эталонное решение, мы можем построить графики распределения плотности, скорости, давления и внутренней энергии по всей области в конечный момент времени $t = T$. 

In [ ]:
fig, axes = plt.subplots(2, 2, sharex='col', num=1)
axes = axes.flatten()
labels = [r'$\rho$', r'$u$', r'$p$', r'$e$']

# For each subplot, plot numerical and exact solutions
for ax, label, num, ref in zip(axes, labels, num_sol, ref_sol[1:]): 
    ax.plot(x, num, 'or', fillstyle='none', label='Numerical')
    ax.plot(ref_sol[0], ref, 'b-', label='Exact')
    ax.set_xlim([0, 1])
    ylim_offset = 0.05 * (np.max(num) - np.min(num))
    ax.set_ylim([np.min(num) - ylim_offset , np.max(num) + ylim_offset])
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(label, rotation=0)
    ax.legend(loc='best')
plt.show()

Переходя слева направо, легко видеть, что наша схема разрешила волну разрежения, разрыв контакта (очевидный в $\rho$ и $e$) и ударную волну. Однако по сравнению с точным решением видно, что численное приближение является размытым и не позволяет точно фиксировать резкие разрывы. Эта неточность, как и ожидалось, относится к схеме, которая является точной только до первого порядка, поскольку термин ошибки второго порядка является диффузионным по своей природе. Каждый раз, когда мы продвигаем систему во времени, численное решение немного размывается по сравнению с точным. После 60 временных шагов результат будет таким, как показано на рисунке. Для любых серьезных применений следует рассматривать схемы с высоким разрешением с точностью не менее второго порядка. Несколько таких схем приведены в [3]. 

## Дополнительные вопросы

* Как вы можете изменить код, чтобы выяснить, сколько шагов было необходимо для достижения $t=T$? 
* Попробуйте увеличить $N$ - такие ли результаты, как вы ожидали? Как это влияет на количество итераций? 
* Каков эффект изменения коэффициента CFL? Существует ли оптимальное значение? 
* Посмотрите, как используются анимации в [Planetary motion](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/planetary_motion_three_body_problem.ipynb). Можете ли вы использовать те же методы для анимации наших результатов с начального времени до конечного состояния? 
* На [странице Википедии](https://en.wikipedia.org/wiki/Sod_shock_tube) для ударной трубки Сода кратко объясняется аналитическое решение, которое приведено в качестве справочного. Как бы вы реализовали точный решатель в Python? 

## References

[1] E. F. Toro: "Riemann Solvers and Numerical Methods for Fluid Dynamics - A Practical Introduction" (3rd ed, Springer, 2009)

[2] E. F. Toro & A. Hidalgo & M. Dumbser: "FORCE schemes on unstructured meshes I: Conservative hyperbolic systems" (Journal of Computational Physics, 2009)

[3] E. F. Toro & S. J. Billett: "Centred TVD Schemes for Hyperbolic Conservation Laws" (IMA Journal of Numerical Analysis, 2000)